## Part 1 - Data Preparation and Exploration 

In [2]:
%%capture
# Due to the configuration of the base Jupter image, the following imports are required for the regressions in the assignment to report the correct metrics

import sys 
!{sys.executable} -m pip uninstall statsmodels --yes 
!{sys.executable} -m pip uninstall numpy --yes
!{sys.executable} -m pip uninstall pandas --yes 
!{sys.executable} -m pip uninstall patsy --yes 
!{sys.executable} -m pip install numpy==1.17
!{sys.executable} -m pip install pandas==1.0
!{sys.executable} -m pip install patsy==0.5.2
!{sys.executable} -m pip install statsmodels==0.11.1

In [3]:
#1 Import Libraries

import pandas as pd
import datetime as dt
import scipy.stats as sp
import numpy as np
import statsmodels.formula.api as sm 

In [4]:
#2 Import Shotlog_14_15 and Player_Stats Datasets

Shotlog_1415=pd.read_csv("Assignment Data/Week 6/Shotlog_14_15.csv")
Player_Stats=pd.read_csv("Assignment Data/Week 6/Player_Stats_14_15.csv")
Shotlog_1415.head()

,game_id,date,match,home_team,away_team,home_away,result,final_margin,shot_number,quarter,...,closest_defender,closest_defender_id,closest_def_dist,current_shot_hit,points_earned,shoot_player,player_id,average_hit,shot_count,shot_per_game
0,21400280,5-Dec-14,ATL @ BKN,BKN,ATL,A,W,23,1,1,...,"Lopez, Brook",201572,6.6,1,2,al horford,201143,0.541259,715,10
1,21400280,5-Dec-14,ATL @ BKN,BKN,ATL,A,W,23,2,1,...,"Lopez, Brook",201572,5.6,0,0,al horford,201143,0.541259,715,10
2,21400280,5-Dec-14,ATL @ BKN,BKN,ATL,A,W,23,3,1,...,"Lopez, Brook",201572,4.7,0,0,al horford,201143,0.541259,715,10
3,21400280,5-Dec-14,ATL @ BKN,BKN,ATL,A,W,23,4,1,...,"Lopez, Brook",201572,5.8,0,0,al horford,201143,0.541259,715,10
4,21400280,5-Dec-14,ATL @ BKN,BKN,ATL,A,W,23,5,2,...,"Lopez, Brook",201572,6.4,0,0,al horford,201143,0.541259,715,10


In [5]:
#3 total number of players
Player_Stats.shape

(281, 2)

In [6]:
#4 total number of shots 
Shotlog_1415.shape

(128069, 27)

In [7]:
#5 Store the date as a date type variable and produce summary stats for variable
Shotlog_1415['date']= pd.to_datetime(Shotlog_1415.date)
Shotlog_1415.date.describe()

count                  128069
unique                    120
top       2015-01-07 00:00:00
freq                     1941
first     2014-10-28 00:00:00
last      2015-03-04 00:00:00
Name: date, dtype: object

In [8]:
#6 Calculate summary stats for shot clock
Shotlog_1415.shot_clock.describe()

count    122502.000000
mean         12.453344
std           5.763265
min           0.000000
25%           8.200000
50%          12.300000
75%          16.675000
max          24.000000
Name: shot_clock, dtype: float64

In [9]:
#7 Create a lagged variable to indicate the result of the previous shot by the same player in same game
#sort the current shot outcome and match
# groupby data by player and match and use shift command to create a lag variable
Shotlog_1415['lag_shot_hit']= Shotlog_1415.sort_values(by=['shot_number'], ascending=True).groupby(['player_id', 'game_id'])['current_shot_hit'].shift(1)
Shotlog_1415.head()
                                                                                                                                                    

,game_id,date,match,home_team,away_team,home_away,result,final_margin,shot_number,quarter,...,closest_defender_id,closest_def_dist,current_shot_hit,points_earned,shoot_player,player_id,average_hit,shot_count,shot_per_game,lag_shot_hit
0,21400280,2014-12-05,ATL @ BKN,BKN,ATL,A,W,23,1,1,...,201572,6.6,1,2,al horford,201143,0.541259,715,10,NaN
1,21400280,2014-12-05,ATL @ BKN,BKN,ATL,A,W,23,2,1,...,201572,5.6,0,0,al horford,201143,0.541259,715,10,1.0
2,21400280,2014-12-05,ATL @ BKN,BKN,ATL,A,W,23,3,1,...,201572,4.7,0,0,al horford,201143,0.541259,715,10,0.0
3,21400280,2014-12-05,ATL @ BKN,BKN,ATL,A,W,23,4,1,...,201572,5.8,0,0,al horford,201143,0.541259,715,10,0.0
4,21400280,2014-12-05,ATL @ BKN,BKN,ATL,A,W,23,5,2,...,201572,6.4,0,0,al horford,201143,0.541259,715,10,0.0


In [10]:
#8 create a variable error to indicate the prediction error for each shot
# create a variable lagerror for the prediciton error for the previous shot
Shotlog_1415['error']= Shotlog_1415.current_shot_hit - Shotlog_1415.average_hit
Shotlog_1415['lagerror']= Shotlog_1415.lag_shot_hit- Shotlog_1415.average_hit
Shotlog_1415.head()

,game_id,date,match,home_team,away_team,home_away,result,final_margin,shot_number,quarter,...,current_shot_hit,points_earned,shoot_player,player_id,average_hit,shot_count,shot_per_game,lag_shot_hit,error,lagerror
0,21400280,2014-12-05,ATL @ BKN,BKN,ATL,A,W,23,1,1,...,1,2,al horford,201143,0.541259,715,10,NaN,0.458741,NaN
1,21400280,2014-12-05,ATL @ BKN,BKN,ATL,A,W,23,2,1,...,0,0,al horford,201143,0.541259,715,10,1.0,-0.541259,0.458741
2,21400280,2014-12-05,ATL @ BKN,BKN,ATL,A,W,23,3,1,...,0,0,al horford,201143,0.541259,715,10,0.0,-0.541259,-0.541259
3,21400280,2014-12-05,ATL @ BKN,BKN,ATL,A,W,23,4,1,...,0,0,al horford,201143,0.541259,715,10,0.0,-0.541259,-0.541259
4,21400280,2014-12-05,ATL @ BKN,BKN,ATL,A,W,23,5,2,...,0,0,al horford,201143,0.541259,715,10,0.0,-0.541259,-0.541259


In [11]:
#9 create summary stats for error and lagshot error 
print(Shotlog_1415.error.describe())
print(Shotlog_1415.lagerror.describe())

count    1.280690e+05
mean    -5.770049e-18
std      4.949640e-01
min     -7.124682e-01
25%     -4.491979e-01
50%     -3.850837e-01
75%      5.395973e-01
max      6.914894e-01
Name: error, dtype: float64
count    113726.000000
mean          0.006303
std           0.496035
min          -0.712468
25%          -0.449198
50%          -0.382143
75%           0.542254
max           0.691489
Name: lagerror, dtype: float64


## Part 2 - Conditional Probability and Autocorrelation

In [12]:
#10 Create dummy bariable conse_shot_hit to indicate a player made consecutive shot hit

Shotlog_1415['conse_shot']= np.where((Shotlog_1415.current_shot_hit ==1) & (Shotlog_1415.lag_shot_hit ==1), 1, 0)
Shotlog_1415.head()

,game_id,date,match,home_team,away_team,home_away,result,final_margin,shot_number,quarter,...,points_earned,shoot_player,player_id,average_hit,shot_count,shot_per_game,lag_shot_hit,error,lagerror,conse_shot
0,21400280,2014-12-05,ATL @ BKN,BKN,ATL,A,W,23,1,1,...,2,al horford,201143,0.541259,715,10,NaN,0.458741,NaN,0
1,21400280,2014-12-05,ATL @ BKN,BKN,ATL,A,W,23,2,1,...,0,al horford,201143,0.541259,715,10,1.0,-0.541259,0.458741,0
2,21400280,2014-12-05,ATL @ BKN,BKN,ATL,A,W,23,3,1,...,0,al horford,201143,0.541259,715,10,0.0,-0.541259,-0.541259,0
3,21400280,2014-12-05,ATL @ BKN,BKN,ATL,A,W,23,4,1,...,0,al horford,201143,0.541259,715,10,0.0,-0.541259,-0.541259,0
4,21400280,2014-12-05,ATL @ BKN,BKN,ATL,A,W,23,5,2,...,0,al horford,201143,0.541259,715,10,0.0,-0.541259,-0.541259,0


In [13]:
#11 create df for the prob of making the previous shot and join prob for making both the previous and current shots

Player_Prob= Shotlog_1415.groupby(['shoot_player'])['conse_shot', 'lag_shot_hit'].mean().reset_index()
Player_Prob.rename(columns={'lag_shot_hit': 'average_lag_hit', 'conse_shot': 'conse_shot_hit'}, inplace=True)
Player_Prob

,shoot_player,conse_shot_hit,average_lag_hit
0,aaron brooks,0.153298,0.418000
1,aaron gordon,0.201923,0.532468
2,al farouq aminu,0.162791,0.465686
3,al horford,0.262937,0.537994
4,al jefferson,0.207500,0.480000
...,...,...,...
276,wesley matthews,0.172460,0.452830
277,wilson chandler,0.141883,0.420741
278,zach lavine,0.156334,0.435737
279,zach randolph,0.229508,0.491143


In [14]:
#12 calculate the conditional probabilty for a player to make a shot given he made the previous 
Player_Prob['conditional_prob']= Player_Prob['conse_shot_hit'] / Player_Prob['average_lag_hit']
Player_Prob.head()

,shoot_player,conse_shot_hit,average_lag_hit,conditional_prob
0,aaron brooks,0.153298,0.418000,0.366741
1,aaron gordon,0.201923,0.532468,0.379221
2,al farouq aminu,0.162791,0.465686,0.349572
3,al horford,0.262937,0.537994,0.488736
4,al jefferson,0.207500,0.480000,0.432292


In [15]:
#13 Merge the Player_Prob dataframe into the Player_Stats dataframe
Player_Stats= pd.merge(Player_Prob, Player_Stats, on='shoot_player')
Player_Stats.head()

,shoot_player,conse_shot_hit,average_lag_hit,conditional_prob,average_hit
0,aaron brooks,0.153298,0.418000,0.366741,0.415330
1,aaron gordon,0.201923,0.532468,0.379221,0.528846
2,al farouq aminu,0.162791,0.465686,0.349572,0.430233
3,al horford,0.262937,0.537994,0.488736,0.541259
4,al jefferson,0.207500,0.480000,0.432292,0.477500


In [16]:
#14 Calculate summary stats for undconditional prob of player making a shot, conditional prob of player making a shot
# given them make the previous one, and prob of player making consectuive shots 

print(Player_Stats.average_hit.describe())
print(Player_Stats.conditional_prob.describe())
print(Player_Stats.conse_shot_hit.describe())

count    281.000000
mean       0.451545
std        0.059392
min        0.308511
25%        0.413223
50%        0.446078
75%        0.480480
max        0.712468
Name: average_hit, dtype: float64
count    281.000000
mean       0.380233
std        0.062320
min        0.225801
25%        0.336689
50%        0.381570
75%        0.422801
max        0.613209
Name: conditional_prob, dtype: float64
count    281.000000
mean       0.176987
std        0.047943
min        0.076190
25%        0.144543
50%        0.171625
75%        0.203512
max        0.422392
Name: conse_shot_hit, dtype: float64


In [17]:
#15 Perform a t-test on the difference between conditional and unconditional probabilites
sp.stats.ttest_ind(Player_Stats.conditional_prob, Player_Stats.average_hit)

Ttest_indResult(statistic=-13.885932802814914, pvalue=6.925846314604593e-38)

In [18]:
#16. Calculate the first order autocorrelation coefficient on making a shot 
Shotlog_1415.current_shot_hit.corr(Shotlog_1415.lag_shot_hit)

-0.010502388301693177

In [19]:
#17. Calculate the first order autocorrelation coefficient on making a shot for each player

Autocorr_Hit= Shotlog_1415.groupby('shoot_player')[['current_shot_hit', 'lag_shot_hit']].corr().unstack().iloc[:,1].reset_index()
Autocorr_Hit.columns= Autocorr_Hit.columns.get_level_values(0)
Autocorr_Hit.rename(columns={'current_shot_hit': 'autocorr'}, inplace=True)
Autocorr_Hit.sort_values(by=['autocorr'], ascending=[False]).head(10)

,shoot_player,autocorr
131,joey dorsey,0.334252
54,cole aldrich,0.174666
200,nate robinson,0.122107
267,tyler hansbrough,0.120608
7,alex len,0.118461
50,cj mccollum,0.115949
114,jason smith,0.105903
190,matt bonner,0.098577
143,jusuf nurkic,0.097465
195,mike miller,0.089366


## Part 3 - Regression Analyses

In [19]:
#18.Reg1: linear least squares regression using the entire shotlog dataframe,
# Dependent variable: error,
# Independent variables:lagerror, shot_dist, dribbles, touch_time, points, quarter, home_away, shoot_player, closest_defender, and closest_def_dist
reg1 = sm.ols(formula = 'error ~ lagerror+shot_dist+dribbles+touch_time+C(points)+C(quarter)+home_away+shoot_player+closest_defender+closest_def_dist', data= Shotlog_1415).fit()
print(reg1.summary())

                            OLS Regression Results                            
Dep. Variable:                  error   R-squared:                       0.057
Model:                            OLS   Adj. R-squared:                  0.051
Method:                 Least Squares   F-statistic:                     9.002
Date:                Thu, 22 Dec 2022   Prob (F-statistic):               0.00
Time:                        19:05:48   Log-Likelihood:                -78065.
No. Observations:              113726   AIC:                         1.577e+05
Df Residuals:                  112960   BIC:                         1.650e+05
Df Model:                         765                                         
Covariance Type:            nonrobust                                         
                                                    coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------------------

In [20]:
#19. Reg2: Weighted least squares regression using the entire shotlog dataframe, weighted by shot_per_game
# Dependent variable: error,
# Independent variables: lagerror, shot_dist, dribbles, touch_time, points, quarter, home_away, shoot_player, closest_defender, and closest_def_dist

reg1 = sm.ols(formula = 'error ~ lagerror+shot_dist+dribbles+touch_time+C(points)+C(quarter)+home_away+closest_def_dist', 
              weights=1/Shotlog_1415.shot_per_game, data= Shotlog_1415).fit()
print(reg1.summary())

                            OLS Regression Results                            
Dep. Variable:                  error   R-squared:                      -9.889
Model:                            OLS   Adj. R-squared:                 -9.891
Method:                 Least Squares   F-statistic:                    -7944.
Date:                Thu, 22 Dec 2022   Prob (F-statistic):               1.00
Time:                        19:18:45   Log-Likelihood:                -79035.
No. Observations:              113726   AIC:                         1.581e+05
Df Residuals:                  113712   BIC:                         1.582e+05
Df Model:                          13                                         
Covariance Type:            nonrobust                                         
                       coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------
Intercept            0.1114      0.005  

In [23]:
#20. Reg3_player: linear least squares on individual players
def Reg3_player(player):
    Shotlog_player=Shotlog_1415[Shotlog_1415.shoot_player==player]
    Reg3_player=sm.ols(formula = 'error ~ lagerror+shot_dist+dribbles+touch_time+C(points)+C(quarter)+home_away+closest_def_dist', data= Shotlog_player).fit()
    print(Reg3_player.summary())
    return

In [24]:
#21. Show regression results for given plyers
Reg3_player('andrew wiggins')

                            OLS Regression Results                            
Dep. Variable:                  error   R-squared:                       0.078
Model:                            OLS   Adj. R-squared:                  0.064
Method:                 Least Squares   F-statistic:                     5.602
Date:                Thu, 22 Dec 2022   Prob (F-statistic):           1.14e-08
Time:                        19:20:39   Log-Likelihood:                -500.04
No. Observations:                 738   AIC:                             1024.
Df Residuals:                     726   BIC:                             1079.
Df Model:                          11                                         
Covariance Type:            nonrobust                                         
                       coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------
Intercept            0.2003      0.057  

In [26]:
Reg3_player('stephen curry')

                            OLS Regression Results                            
Dep. Variable:                  error   R-squared:                       0.074
Model:                            OLS   Adj. R-squared:                  0.062
Method:                 Least Squares   F-statistic:                     6.498
Date:                Thu, 22 Dec 2022   Prob (F-statistic):           1.79e-10
Time:                        19:21:51   Log-Likelihood:                -625.19
No. Observations:                 910   AIC:                             1274.
Df Residuals:                     898   BIC:                             1332.
Df Model:                          11                                         
Covariance Type:            nonrobust                                         
                       coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------
Intercept            0.2196      0.057  

In [28]:
## 22.Reg4_wls_player: weighted least squares regressions on individual players, weighted by shot_per_game.
def Reg4_wls_player(player):
    Shotlog_player=Shotlog_1415[Shotlog_1415.shoot_player==player]
    Reg4_wls_player=sm.wls(formula = 'error ~ lagerror+shot_dist+dribbles+touch_time+C(points)+C(quarter)+home_away+closest_def_dist',weights=1/Shotlog_player['shot_per_game'] , data= Shotlog_player).fit()
    print(Reg4_wls_player.summary())
    return

In [31]:
Reg4_wls_player('andrew wiggins')

                            WLS Regression Results                            
Dep. Variable:                  error   R-squared:                       0.073
Model:                            WLS   Adj. R-squared:                  0.059
Method:                 Least Squares   F-statistic:                     5.179
Date:                Thu, 22 Dec 2022   Prob (F-statistic):           7.18e-08
Time:                        19:23:25   Log-Likelihood:                -541.82
No. Observations:                 738   AIC:                             1108.
Df Residuals:                     726   BIC:                             1163.
Df Model:                          11                                         
Covariance Type:            nonrobust                                         
                       coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------
Intercept            0.1465      0.056  

In [29]:
Reg4_wls_player('stephen curry')

                            WLS Regression Results                            
Dep. Variable:                  error   R-squared:                       0.078
Model:                            WLS   Adj. R-squared:                  0.066
Method:                 Least Squares   F-statistic:                     6.868
Date:                Thu, 22 Dec 2022   Prob (F-statistic):           3.42e-11
Time:                        19:22:57   Log-Likelihood:                -660.11
No. Observations:                 910   AIC:                             1344.
Df Residuals:                     898   BIC:                             1402.
Df Model:                          11                                         
Covariance Type:            nonrobust                                         
                       coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------
Intercept            0.2125      0.057  